## Do polar plots of all records in selected day

In [7]:
using CSV, CairoMakie
using Dates, DataFrames, DSP
using GLMakie
using NativeFileDialog
using StatsBase


function smooth_spectra(Pden_in, sample_frequency)
##################################################
# smooth the spectra into bands centered on 0.05Hz spacing (i.e. 0:0.005:0.64)
    nyquist = sample_frequency/2

    freq_in = range(0, stop=nyquist, length=length(Pden_in))

    freq_out = [0.0]
    Pden_smoothed = [mean(Pden_in[1:8])]

    i = 9
    while i <= length(Pden_in)

        push!(freq_out,freq_in[i+8])

        if i < length(Pden_in)-16

            push!(Pden_smoothed, mean(Pden_in[i:i+16]))

        end

        i+=16

    end

    push!(Pden_smoothed, mean(Pden_in[end-8:end]))
            
    return (freq_out, Pden_smoothed)
        
end    # smooth_spectra()


function get_Fourier_coefficients(heave, north, west)
#####################################################    
    # Get the cross periodograms
    cps_heave_heave = mt_cross_power_spectra([heave heave]', fs=sample_frequency);
    cps_north_north = mt_cross_power_spectra([north north]', fs=sample_frequency);
    cps_west_west = mt_cross_power_spectra([west west]', fs=sample_frequency);

    cps_north_heave = mt_cross_power_spectra([north heave]', fs=sample_frequency);
    cps_west_heave = mt_cross_power_spectra([west heave]', fs=sample_frequency);
    cps_north_west = mt_cross_power_spectra([north west]', fs=sample_frequency);

##    fhh = cps_heave_heave.freq
    fhh, Chh = smooth_spectra(real.(cps_heave_heave.power[1,1,:]), sample_frequency)

    #fnn = cps_north_north.freq
    fhh, Cnn = smooth_spectra(real.(cps_north_north.power[1,1,:]), sample_frequency)

    #fww = cps_west_west.freq
    fhh, Cww = smooth_spectra(real.(cps_west_west.power[1,1,:]), sample_frequency)

    #fnw = cps_north_west.freq
    fhh, Cnw = smooth_spectra(real.(cps_north_west.power[1,1,:]), sample_frequency)

    #fnh = cps_north_heave.freq
    fhh, Qnh = smooth_spectra(imag.(cps_north_heave.power[1,2,:]), sample_frequency)

    #fwh = cps_west_heave.freq
    fhh, Qwh = smooth_spectra(imag.(cps_west_heave.power[1,2,:]), sample_frequency)

    a1 = Qnh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5
    b1 = -Qwh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5

    a2 = (Cnn .- Cww) ./ (Cnn .+ Cww)
    b2 = -2 .* Cnw ./ (Cnn .+ Cww)
    
    return(fhh, Chh, a1, b1, a2, b2)
    
end    # get_Fourier_coefficients()


function get_spec_dir(displacement_df, total)
#############################################
        
    Chh = displacement_df.Chh[total]
    a1 = displacement_df.a1[total]
    b1 = displacement_df.b1[total] 
    a2 = displacement_df.a2[total] 
    b2 = displacement_df.b2[total]
    time_string = displacement_df.Time_string[total]

    aa = length(Chh) # Number of spectral points

    r = 1:6:aa
    ρ = r ./ (aa/nyquist) 

    θ = 0:pi/180:2pi

    # populate a matrix of spectral surface values
    mat = [Chh[r1] * (a1[r1]*cos.(θ) + b1[r1]*sin.(θ) + a2[r1]*cos.(2θ) + b2[r1]*sin.(2θ)) for r1 in r]
    mat = hvcat(size(mat,1),mat...)
        
    # set any values less than zero to zero
    mat[mat .< 0] .= 0
    
    return(θ, ρ, mat, time_string)
    
end    # get_spec_dir()
   

function fix_length(wse)
########################
    
    length_wse = length(wse)
    if length_wse < 4608
        append!(wse,zeros(4608-length_wse+1))
    end

    return(wse)

    end

        
###############################################
################################################
################################################
##           START OF MAIN PROGRAM
################################################
################################################
################################################

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# Select a CSV file
infil = pick_file("C:\\QGHL\\Wave_data\\", filterlist="CSV,csv");
println("Selected ",infil)

if uppercase(split(infil, ".")[end]) == "CSV"

    # read .csv file to df
    println("Reading data from file to dataframe")
    flush(stdout)
    df = DataFrame(CSV.File(infil,header=0, delim="\t"));
    
    # name df headers
    rename!(df,[:Date,:Status, :Heave, :North, :West]);

    # convert Epoch seconds to UTC datetime
    df.Date = unix2datetime.(df.Date);
    
else

    println("Not able to read this file type at present")
    flush(stdout)
    exit()

end

# get date of .csv file
file_date = DateTime(last(split(first(infil, length(infil)-4),"}")))
first_date = file_date; last_date = first_date + Day.(1)

println("Preparing plots")
flush(stdout)
title_string = Dates.format.(file_date,"dd/mm/yyyy")

const sample_frequency = 2.56
nyquist = sample_frequency/2

# select rows for the day of interest only (this discards the 23:30 record from previous day)
day_df = df[findall(day.(df.Date) .== mode(day.(df.Date))),:]
    
# Build daily df containing spectral data
record = 1
start_val = DateTime(Date(day_df.Date[1]))
end_val = start_val + Minute(30)
    
# build df containing displacements and Fourier coefficient for selected day
displacement_df = DataFrame(Time_string = [], Heave = [], North = [], West = [], fhh = [], Chh = [], a1 = [], b1 = [], a2 = [], b2 = [], mat = [])

println("Processing each 30-minute record for selected day")

while record <= 48
        
    if (mod(record,10) == 0)
        print(string(record))
    else
        print(".")
    end
    
    try
        # find all records for a 30-minute record
        temp_df = day_df[start_val .<= day_df.Date .< end_val,:]
        heave, north, west = temp_df.Heave, temp_df.North, temp_df.West
                        
        heave = fix_length(heave)
        north = fix_length(north)
        west = fix_length(west)
            
        # calculate Fourier coefficients
        fhh, Chh, a1, b1, a2, b2 = get_Fourier_coefficients(heave, north, west)
        
        aa = length(Chh) # Number of spectral points
        
        r = 1:6:aa
        global ρ = r ./ (aa/nyquist) 

        global θ = 0:pi/180:2pi        

        # populate a matrix of spectral surface values
        mat = [Chh[r1] * (a1[r1]*cos.(θ) + b1[r1]*sin.(θ) + a2[r1]*cos.(2θ) + b2[r1]*sin.(2θ)) for r1 in r]
                    
        mat = hvcat(size(mat,1),mat...)

        # set any values less than zero to zero
        mat[mat .< 0] .= 0
        
        time_string = Dates.format(start_val, "dd/mm/yyyy HH:MM")

        # add spectral data to plot df
        push!(displacement_df, (time_string, heave, north, west, fhh, Chh, a1, b1, a2, b2, mat));
            
        record += 1
        start_val = end_val
        end_val += Minute(30)

    catch
        
        println("Error")
        break
        
    end
    
end

println("\nPlotting 30-minute records now")
flush(stdout)

# get the highest energy value for the day
# this sets scaling of plots                    
spec_max = maximum(maximum.(displacement_df.mat))
                    
println("Maximum spectra for day = ",round(spec_max, digits=2),"m²/Hz.")
                    
# declare the Observables
inc = Observable(1)
time_string = Observable(displacement_df[inc[],:].Time_string)
mat = Observable(Float64.(displacement_df[inc[],:].mat))
fhh_L = Observable(Int(round(length(displacement_df.Chh[1]) / 6)))
        
fig = CairoMakie.Figure(size=(800, 850))

# draw the polar axis
ax = CairoMakie.PolarAxis(fig[1, :],
    thetaticklabelsize = 15,  
    rlimits=(0,0.4), rticklabelsize=15, rticks=0:0.2:0.4, rgridwidth=0.5, rtickangle=180, rminorgridvisible=true, rminorgridstyle=:dot,
    theta_0=-pi/2, thetagridwidth=0.5, thetaminorgridvisible=true, thetaminorgridstyle=:dot, thetaminorticks=IntervalsBetween(3), 
    direction=-1, width=630, height=610, title=time_string, titlesize=24,
    )

# Set plotting values
cmap = Reverse(:ocean)
levels = round(spec_max/100, digits=2):round(spec_max/20, digits=2):round(spec_max, digits=2)
θ = 0:pi/180:2pi
ρ = range(0, stop=1.28, length=fhh_L[])

# do contour plot
c1 = CairoMakie.contourf!(ax, θ, ρ, mat, colormap=cmap, levels=levels)
c1 = CairoMakie.contour!(ax, θ, ρ, mat, colormap=cmap, levels=levels)
                    
ax = CairoMakie.Colorbar(fig[2, :], limits=(0, round(spec_max, digits=1, RoundUp)), label="Spectral Density (m²/Hz.)", labelsize=:20, 
            width=500, height=30, vertical=false, flipaxis=false, colormap=cmap)  

display(fig) 

# update the Observables
for i in 1:nrow(displacement_df)

    inc[] = i
    time_string[] = displacement_df[inc[],:].Time_string

    try
        mat[] = Float64.(displacement_df.mat[inc[]])
    catch
        break
    end

    sleep(0.3)
    yield()

end
                            
###############################################
################################################
################################################
##           END OF MAIN PROGRAM
################################################
################################################
################################################
                            

Selected D:\brisbane_4183\2023\12\brisbane_4183{disp}2023-12-18.csv
Reading data from file to dataframe
Preparing plots
Processing each 30-minute record for selected day
.........10.........20.........30.........40........
Plotting 30-minute records now
Maximum spectra for day = 3.96m²/Hz.


In [9]:
frames = 1:nrow(displacement_df)

GLMakie.record(fig, ".\\Brisbane_polar1_animation.mp4", frames;
        framerate=6) do frame
    
    time_string[] = displacement_df.Time_string[frame]
    mat[] = Float64.(displacement_df.mat[frame])
    
    if (mod(frame,6) == 0)
        print(string(frame))
    else
        print(".")
    end
                  
end

.....6.....12.....18.....24.....30.....36.....42.....48

".\\Brisbane_polar1_animation.mp4"